In [4]:
#load "Paket.fsx"
Paket.Package
  [ "QBitNinja.Client -Version 1.0.3.46"
    "NBitcoin -Version 4.0.0.48"
    "FSharp.Data -Version 2.4.6"
    "Newtonsoft.Json -Version 10.0.3"    
  ]    
#load "Paket.Generated.Refs.fsx"

open System
open NBitcoin
open NBitcoin.Crypto   
open NBitcoin.OpenAsset 
open NBitcoin.Stealth 
open QBitNinja.Client 
open QBitNinja.Client.Models 
open System.Collections.Generic 
open System.Linq
open System.Collections   

open FSharp.Data
open FSharp.Data.JsonExtensions

type secretJson = JsonProvider<"""{
  "secret": ""
}""">
      
let sj = secretJson.GetSample()

In [5]:
let alice = secretJson.Load(@"/home/nbuser/library/secret.json").JsonValue?secret.AsString()  
let bob = secretJson.Load(@"/home/nbuser/library/secret_dest.json").JsonValue?secret.AsString()  
let zorro = secretJson.Load(@"/home/nbuser/library/secret_zorro.json").JsonValue?secret.AsString()  
  
let happy = secretJson.Load(@"/home/nbuser/library/secret_zorro.json").JsonValue?secret.AsString() 

let alice_private  = Key.Parse(alice, Network.TestNet) 
let bob_private  = Key.Parse(bob, Network.TestNet) 


In [6]:
let client:QBitNinjaClient  = new QBitNinjaClient(Network.TestNet)

let transactionId = uint256.Parse("d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b")

// Query the transaction
let transactionResponse:GetTransactionResponse  = client.GetTransaction(transactionId).Result

let transaction:NBitcoin.Transaction  = transactionResponse.Transaction

let receivedCoins = transactionResponse.ReceivedCoins;
let mutable myCoins:Coin = null 
for coin in receivedCoins do
    if (coin.TxOut.ScriptPubKey =  alice_private.ScriptPubKey) then
        myCoins<- coin:?>Coin
        
printfn "%A" myCoins.Amount

0.53990000


In [4]:
let issuanceCoins':IssuanceCoin[]  = [|new IssuanceCoin(myCoins)|]

issuanceCoins' |> Util.Table

AssetId,DefinitionUrl,Amount,TxOut,ScriptPubKey,Bearer,Outpoint,CanGetScriptCode
e698a026e7a9cc32002df6d2478bd78425d5b3aa,,0.53990000,NBitcoin.TxOut,OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG,NBitcoin.Coin,d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b-0,True


In [7]:
let mutable tx_out:TxOut = new TxOut(new Money(0.1m, MoneyUnit.BTC),alice_private.PubKey)
let mutable tx_out':TxOut = new TxOut(new Money(0.1m, MoneyUnit.BTC),alice_private.PubKey)

let mutable issuanceCoinsTransaction  = new Transaction()
issuanceCoinsTransaction.Outputs.Add(tx_out)
issuanceCoinsTransaction.Outputs.Add(tx_out')

let issuanceCoin = new Coin(issuanceCoinsTransaction,tx_out)

let goldIssuanceCoin = new IssuanceCoin(issuanceCoin) 

goldIssuanceCoin.DefinitionUrl <- new Uri( @"https://zorrocomp.org/la_mia_nota_legale.html")

Typically such coin has a BTC amount **more or less equals to 600 Satoshi**

In [9]:

let goldId = goldIssuanceCoin.AssetId

printfn "%A" goldId

9a6b0084085fbb87711768a472f8407bbee718b7


[Open Assets Protocol](https://github.com/OpenAssets/open-assets-protocol)

In [11]:

let mutable txBuilder = new TransactionBuilder() 
let mutable tx:Transaction = txBuilder
                                .AddCoins(goldIssuanceCoin) 
                                .AddKeys(alice_private)
                                .IssueAsset(bob_private, new AssetMoney(goldId, 30))
                                .SetChange(alice_private)
                                .BuildTransaction(true)

In [12]:
printfn "%A" tx

{
  "hash": "d125871a223e28aa4dc0de682c72822e70eaa24316e4e8812e9ce20d2f71ad72",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 3,
  "lock_time": 0,
  "size": 257,
  "in": [
    {
      "prev_out": {
        "hash": "5923b983626f91c9753d1953523504cac32fcc044b2eb55435dcad49ef9ce671",
        "n": 0
      },
      "scriptSig": "30450221008b0e33a2dc2e0373ca0c9cb01eddaf0224563a82100e689599b8ef3b468ee94d022027e0bbe3ab21e5ed4bd66f91a6cf6fc6dc1cbb877605aae975fe161aa98690ea01"
    }
  ],
  "out": [
    {
      "value": "0.00000546",
      "scriptPubKey": "OP_DUP OP_HASH160 8efb2874941ae0a85e2c94ee6b267e8ad70fb5cf OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.09999454",
      "scriptPubKey": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.00000000",
      "scriptPubKey": "OP_RETURN 4f410100011e2f753d68747470733a2f2f7a6f72726f636f6d702e6f72672f6c615f6d69615f6e6f74615f6c6567616c652e68746d6c"
    }
  ]
}


In [13]:
let mutable txRepo = new NoSqlTransactionRepository()
txRepo.Put(issuanceCoinsTransaction.GetHash(), issuanceCoinsTransaction)
let mutable ctxRepo = new NoSqlColoredTransactionRepository(txRepo)

In [14]:
let ctx = tx.GetColoredTransaction(ctxRepo)

In [15]:
printfn "%A" ctx


{
  "inputs": [],
  "issuances": [
    {
      "index": 0,
      "asset": "AVrMtfquooTZtSQzEZWb4e6nanNq8Dn8Pz",
      "quantity": 30
    }
  ],
  "transfers": [],
  "destructions": []
}


In [ ]:
(article)[https://coinprism.desk.com/customer/en/portal/articles/1682916-how-do-i-do-a-crowdsale-with-coinprism-]